In [2]:
from recipe_nlg import RecipeNLGDataset, TokenizedRecipeNLGDataset
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import kagglehub

BATCH_SIZE = 16

path = kagglehub.dataset_download("paultimothymooney/recipenlg")
# Load the dataset
df = pd.read_csv(path + "/RecipeNLG_dataset.csv", header=0)
# Create an instance of the RecipeNLGDataset class



In [14]:
from pathlib import Path
from transformers import PreTrainedTokenizerFast

tokenizer_path = Path("title_to_all_tokenizer")
print("Loading tokenizer")
hf_tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path, model_max_lenth=512)

Loading tokenizer


In [ ]:
# 'all' is default mode
data = RecipeNLGDataset(df)

In [38]:
recipe_batch = data.recipe_strings[:16]

tokenized_batch = hf_tokenizer(
    text=recipe_batch.tolist(),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

tokenized_batch


{'input_ids': tensor([[ 1146,    12,   300,  ...,     0,     0,     0],
        [13024,    66,   842,  ...,     0,     0,     0],
        [ 1425,   442,     2,  ...,     0,     0,     0],
        ...,
        [ 1767,   610,   679,  ...,     0,     0,     0],
        [ 1725,  1183,     9,  ...,     0,     0,     0],
        [  327,  1604,   348,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [41]:
proper_format = {
    "input_ids": tokenized_batch["input_ids"].squeeze(0),
    "attention_mask": tokenized_batch["attention_mask"].squeeze(0),
    # etc.
}

print(proper_format["attention_mask"].shape)
proper_format["input_ids"].shape


torch.Size([16, 512])


torch.Size([16, 512])

In [43]:
from datasets import Dataset


hf_ds = Dataset.from_dict({
    k: v.numpy()  # Datasets accepts numpy arrays
    for k, v in proper_format.items()
})

print(hf_ds)




Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 16
})


In [ ]:
import torch
# Custom collate function
def collate_fn(batch):
    # Convert each field in the batch to a PyTorch tensor
    return {
        key: torch.stack([torch.tensor(item[key]) for item in batch])
        for key in batch[0]
    }

loader = DataLoader(hf_ds, batch_size=4, shuffle=True, collate_fn=collate_fn)

for i, batch in enumerate(loader):
    print(batch['input_ids'].shape) # (batch size, max_length) -> this works

torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])


In [ ]:
# ensure decoding works
first_batch = next(iter(loader))
first_example = first_batch['input_ids'][0]
hf_tokenizer.decode(first_example)




'rhubarb coffee cake <end_title> 1 12 c . sugar , 12 c . butter , 1 egg , 1 c . buttermilk , 2 c . flour , 12 tsp . salt , 1 tsp . soda , 1 c . buttermilk , 2 c . rhubarb , finely cut , 1 tsp . vanilla <end_ingredients> cream sugar and butter . add egg and beat well . to creamed butter , sugar and egg , add alternately buttermilk with mixture of flour , salt and soda . mix well . add rhubarb and vanilla . pour into greased 9 x 13 - inch pan and add topping . <end> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [77]:
import torch.nn as nn

input_ids = first_batch['input_ids']

vocab_size = len(hf_tokenizer.get_vocab())
d_model = 65
context_length = len(input_ids)



# embedding layer: one row for every token in vocab, embedding-size columns
embedding = nn.Embedding(vocab_size, d_model)


x = embedding(input_ids)
x.shape # (batchsize x seq_length x embed dim)

torch.Size([4, 512, 65])

In [83]:
from PositionalEncoder import PositionalEncoder


In [84]:
pe = PositionalEncoder(context_len=context_length, d_model=d_model)

AttributeError: module 'torch' has no attribute 'arrange'